In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [21]:
url = 'https://raw.githubusercontent.com/COGS118A/Group009-SP23/main/clean_data.csv'
data = pd.read_csv(url)


In [23]:

# Ensure data is loaded correctly
data = data.dropna()
print(data.shape)
#print(data.isnull().sum())


(16467, 216)


In [27]:


# Define predictors and labels
X = data.drop('good_outcome', axis=1)
y = data['good_outcome']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the model
model = LogisticRegression(max_iter=1000)

# Fit the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)


In [30]:
# Evaluate the model
print("Model accuracy: ", accuracy_score(y_test, y_pred, normalize=True), "\n")

print("Classification report:")
print(classification_report(y_test, y_pred))

Model Accuracy:  0.8443634891722324 

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.69      0.77      1828
           1       0.84      0.93      0.88      3113

    accuracy                           0.84      4941
   macro avg       0.85      0.81      0.83      4941
weighted avg       0.85      0.84      0.84      4941

